In [1]:
import numpy as np 
import pandas as pd

In [4]:
books=pd.read_csv('Books.csv',on_bad_lines="skip",sep=';')
users=pd.read_csv("Users.csv",on_bad_lines="skip",sep=';')
ratings=pd.read_csv("Ratings.csv",on_bad_lines="skip",sep=';')

/tmp/ipykernel_5825/414528685.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users=pd.read_csv("Users.csv",on_bad_lines="skip",sep=';')


In [24]:
import string

def ascii_check(item):
    """
    Checks if a given item contains only ASCII characters.

    Args:
        item: The item to check.

    Returns:
        1 if the item contains non-ASCII characters, 0 otherwise.
    """
    for letter in str(item):
        if letter not in string.printable:  # Use string.printable for all ASCII characters
            return 1
    return 0

def ascii_check_bulk(df):
    """
    Checks for non-ASCII characters in all columns of a DataFrame.

    Args:
        df: The DataFrame to check.
    """
    for col in df.columns:
        print(f"Items with non-ASCII characters in {col}: {df[col].apply(ascii_check).sum()}")
    print('')

In [25]:
def colname_fix(colname):
    return colname.lower().replace('-','_')

In [26]:
for df in [books,users,ratings]:
    df.columns=[colname_fix(col) for col in df.columns]

In [27]:
print('Ratings:\nNumber of ratings: %d\nNumber of books: %d\nNumber of users: %d' % (len(ratings),
                                                                                     len(ratings['isbn'].unique()),
                                                                                     len(ratings['user_id'].unique())))
print('\nNumber of books: %d' % len(books))
print('\nNumber of users: %d' % len(users))

Ratings:
Number of ratings: 1149780
Number of books: 340556
Number of users: 105283

Number of books: 271379

Number of users: 278859


In [28]:
ascii_check_bulk(ratings)
ascii_check_bulk(users)
ascii_check_bulk(books)

Items with non-ASCII characters in user_id: 0
Items with non-ASCII characters in isbn: 17
Items with non-ASCII characters in rating: 0

Items with non-ASCII characters in user_id: 0
Items with non-ASCII characters in age: 4

Items with non-ASCII characters in isbn: 0
Items with non-ASCII characters in title: 6634
Items with non-ASCII characters in author: 2086
Items with non-ASCII characters in year: 0
Items with non-ASCII characters in publisher: 1796



In [23]:
books.shape

(271379, 5)

In [30]:
ratings['isbn_check'] = ratings['isbn'].apply(ascii_check)
ratings = ratings[ratings['isbn_check']==0]

In [31]:
ratings.drop(['isbn_check'], axis=1, inplace=True)

/tmp/ipykernel_5825/2125850976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.drop(['isbn_check'], axis=1, inplace=True)


In [36]:
df_ratings_explicit = ratings[ratings['rating']!=0]
df_ratings_implicit = ratings[ratings['rating']==0]

print('Explicit ratings: %d\nImplicit ratings: %d' % (len(df_ratings_explicit), len(df_ratings_implicit)))


Explicit ratings: 433657
Implicit ratings: 716106


In [38]:

books.to_csv('books.csv', encoding='utf-8', index=False)
users.to_csv('users.csv', encoding='utf-8', index=False)
df_ratings_explicit.to_csv('explicit_ratings.csv', encoding='utf-8', index=False)
df_ratings_implicit.to_csv('implicit_rating.csv',encoding='utf-8',index=False)



In [42]:
user_rating_thresold=3

filter_user=df_ratings_explicit['user_id'].value_counts()
filter_user_list=filter_user[filter_user>=user_rating_thresold].index.to_list()
df_ratings_top = df_ratings_explicit[df_ratings_explicit['user_id'].isin(filter_user_list)]

print('Filter: users with at least %d ratings\nNumber of records: %d' % (user_rating_thresold, len(df_ratings_top)))

Filter: users with at least 3 ratings
Number of records: 368564


In [41]:
filter_user

user_id
11676     8523
98391     5802
153662    1969
189835    1906
23902     1395
          ... 
114076       1
114079       1
114081       1
114096       1
276723       1
Name: count, Length: 77803, dtype: int64

In [43]:
book_ratings_threshold_perc = 0.1
book_ratings_threshold = len(df_ratings_top['isbn'].unique()) * book_ratings_threshold_perc

filter_books_list = df_ratings_top['isbn'].value_counts().head(int(book_ratings_threshold)).index.to_list()
df_ratings_top = df_ratings_top[df_ratings_top['isbn'].isin(filter_books_list)]

print('Filter: top %d%% most frequently rated books\nNumber of records: %d' % (book_ratings_threshold_perc*100, len(df_ratings_top)))

Filter: top 10% most frequently rated books
Number of records: 176595


In [45]:
df_ratings_top.to_csv('ratings_top.csv', encoding='utf-8', index=False)